In [1]:
# Warehouse AI Monitoring System - YOLOv12 Training Pipeline
# Author: AI Assistant
# Description: Train YOLOv12 model on warehouse datasets for object detection and counting

import os
import shutil
import yaml
from pathlib import Path

# Install required packages
!pip install --no-deps roboflow ultralytics


print("📦 Packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
📦 Packages installed successfully!


In [2]:
!pip install pillow_heif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.1.64 requires filetype, which is not installed.
roboflow 1.1.64 requires python-dotenv, which is not installed.
roboflow 1.1.64 requires idna==3.7, but you have idna 3.10 which is incompatible.
roboflow 1.1.64 requires opencv-python-headless==4.10.0.84, but you have opencv-python-headless 4.11.0.86 which is incompatible.


In [3]:
!pip install filetype

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.1.64 requires python-dotenv, which is not installed.
roboflow 1.1.64 requires idna==3.7, but you have idna 3.10 which is incompatible.
roboflow 1.1.64 requires opencv-python-headless==4.10.0.84, but you have opencv-python-headless 4.11.0.86 which is incompatible.


In [4]:
!pip install dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.1.64 requires idna==3.7, but you have idna 3.10 which is incompatible.
roboflow 1.1.64 requires opencv-python-headless==4.10.0.84, but you have opencv-python-headless 4.11.0.86 which is incompatible.


In [5]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

CUDA available: True
Device count: 1
GPU name: Tesla P100-PCIE-16GB


In [6]:
# Warehouse AI Monitoring System - YOLO Training Pipeline
# Author: AI Assistant (Modified by User, Corrected by Gemini)
# Description: Train YOLO model on warehouse datasets with unified class handling



# =============================================================================
# STEP 1: Download and Setup Datasets
# =============================================================================

from roboflow import Roboflow

def download_datasets():
    print("\U0001F504 Downloading Dataset 1: Warehouse General...")
    dataset1 = None # Initialize to None
    try:
        # IMPORTANT: Replace "YOUR_ROBOFLOW_API_KEY" with your actual Roboflow API key
        # or use environment variables for better security.
        rf1 = Roboflow(api_key="yZoots1smL4YKB00QFuC") # Replace with your key or os.environ.get("ROBOFLOW_API_KEY")
        project1 = rf1.workspace("blueberry-jtbtk").project("detection_box-q7cw4")
        version1 = project1.version(1)
        dataset1 = version1.download("yolov8")
        print(f"\u2705 Dataset 1 downloaded to: {dataset1.location}")
    except Exception as e:
        print(f"\u26A0\uFE0F Error downloading dataset 1: {e}")
        dataset1 = None

    print("\U0001F504 Downloading Dataset 2: Warehouse Object Detection...")
    dataset2 = None # Initialize to None
    try:
        # IMPORTANT: Replace "YOUR_ROBOFLOW_API_KEY" with your actual Roboflow API key
        rf2 = Roboflow(api_key="yZoots1smL4YKB00QFuC") # Replace with your key or os.environ.get("ROBOFLOW_API_KEY")
        project2 = rf2.workspace("veeck").project("vikas_warehouse_obj_detection")
        version2 = project2.version(10)
        dataset2 = version2.download("yolov8")
        print(f"\u2705 Dataset 2 downloaded to: {dataset2.location}")
    except Exception as e:
        print(f"\u26A0\uFE0F Error downloading dataset 2: {e}")
        dataset2 = None

    if dataset1 is None and dataset2 is None:
        raise RuntimeError("\u274C Failed to download any datasets!")

    print("\u2705 Dataset download completed!")
    return dataset1, dataset2

# Potentially run download if datasets are not already available
# dataset1_obj, dataset2_obj = download_datasets()
# For testing without re-downloading, you might manually set these paths if already downloaded:
# from types import SimpleNamespace
# dataset1_obj = SimpleNamespace(location='./Warehouse-General-1') # Example path
# dataset2_obj = SimpleNamespace(location='./Vikas_Warehouse_Obj_Detection-10') # Example path
# Ensure these paths exist and contain the data.yaml and image/label folders.
# For now, we'll call the download function.
dataset1_obj, dataset2_obj = download_datasets()

# =============================================================================
# STEP 2: Combine Datasets
# =============================================================================

def combine_datasets(ds1_obj, ds2_obj):
    print("\U0001F504 Combining datasets...")

    combined_path = Path("./combined_warehouse_dataset")
    for split in ['train', 'valid', 'test']:
        (combined_path / split / 'images').mkdir(parents=True, exist_ok=True)
        (combined_path / split / 'labels').mkdir(parents=True, exist_ok=True)

    dataset_locations = []
    if ds1_obj and hasattr(ds1_obj, 'location') and Path(ds1_obj.location).exists():
        dataset_locations.append(Path(ds1_obj.location))
    else:
        print("Dataset 1 location is invalid or not found. Skipping.")

    if ds2_obj and hasattr(ds2_obj, 'location') and Path(ds2_obj.location).exists():
        dataset_locations.append(Path(ds2_obj.location))
    else:
        print("Dataset 2 location is invalid or not found. Skipping.")

    if not dataset_locations:
        print("\U0001F6AB No valid datasets to combine.")
        return None, []

    all_classes = set()
    processed_dataset_info = []

    for loc_path in dataset_locations:
        yaml_path = loc_path / "data.yaml"
        if yaml_path.exists():
            with open(yaml_path, 'r') as f:
                data = yaml.safe_load(f)
                names = data.get('names', [])
                if isinstance(names, list) and all(isinstance(item, str) for item in names): # Standard list of names
                    pass
                elif isinstance(names, dict): # Dictionary like {0: 'name1', 1: 'name2'}
                    names = list(names.values())
                else:
                    print(f"\u26A0\uFE0F Warning: Class names in {yaml_path} are not in expected list or dict format. Skipping this dataset's classes for unification.")
                    continue # Skip if names format is unexpected for class processing
                
                names = [str(n).lower() for n in names]  # Normalize class names to lowercase strings
                processed_dataset_info.append({'path': loc_path, 'class_map': names})
                all_classes.update(names)
        else:
            print(f"\u26A0\uFE0F Warning: data.yaml not found in {loc_path}. Skipping this dataset for combination.")


    if not all_classes:
        print("\U0001F6AB No classes found in any dataset. Cannot combine.")
        return None, []

    unified_classes = sorted(list(all_classes))

    def remap_label(label_path, class_map_old_dataset, current_unified_classes):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            try:
                old_cls_idx = int(parts[0])
                if 0 <= old_cls_idx < len(class_map_old_dataset):
                    class_name_original = class_map_old_dataset[old_cls_idx] # Already lowercased during collection
                    if class_name_original in current_unified_classes:
                        new_cls_idx = current_unified_classes.index(class_name_original)
                        new_line = ' '.join([str(new_cls_idx)] + parts[1:])
                        new_lines.append(new_line)
                    else:
                        print(f"\U0001F6AB Warning: Class '{class_name_original}' from {label_path} not in unified_classes. Skipping line.")
                else:
                    print(f"\U0001F6AB Warning: Invalid class index {old_cls_idx} in {label_path}. Skipping line.")
            except ValueError:
                print(f"\U0001F6AB Warning: Malformed line in {label_path}: {line.strip()}. Skipping line.")
            except IndexError:
                 print(f"\U0001F6AB Warning: Index error for class mapping in {label_path}. Old class map length: {len(class_map_old_dataset)}, index: {old_cls_idx}. Skipping line.")
        return new_lines

    file_count = 0
    for i, ds_info in enumerate(processed_dataset_info):
        dataset_path = ds_info['path']
        class_map_old = ds_info['class_map']
        # Use a unique prefix, e.g., based on dataset folder name if possible, or stick to ds{i+1}
        prefix = f"{dataset_path.name.replace(' ', '_')}_" # Make prefix from dataset folder name

        for split in ['train', 'valid', 'test']:
            src_img_dir = dataset_path / split / 'images'
            src_lbl_dir = dataset_path / split / 'labels'
            
            if not src_img_dir.exists():
                # print(f"Info: Image directory not found for {split} in {dataset_path}, skipping.")
                continue
            if not src_lbl_dir.exists():
                # print(f"Info: Label directory not found for {split} in {dataset_path}, skipping image copying for this split as labels are crucial.")
                # continue # If labels must exist, uncomment this
                pass # If images without labels are okay, comment above and keep pass

            for img_file in src_img_dir.glob('*'):
                if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp', '.webp']:
                    dst_img_name = f"{prefix}{img_file.name}"
                    dst_img = combined_path / split / 'images' / dst_img_name
                    shutil.copy2(img_file, dst_img)
                    file_count += 1

                    lbl_file_name = f"{img_file.stem}.txt"
                    lbl_file = src_lbl_dir / lbl_file_name
                    if lbl_file.exists():
                        remapped_lines = remap_label(lbl_file, class_map_old, unified_classes)
                        if remapped_lines: # Only write if there's content
                            dst_lbl_name = f"{prefix}{img_file.stem}.txt"
                            dst_lbl = combined_path / split / 'labels' / dst_lbl_name
                            with open(dst_lbl, 'w') as f:
                                f.write('\n'.join(remapped_lines))
                        # else: # Optional: print if a label file becomes empty after remapping
                        #     print(f"Info: Label file {lbl_file} resulted in no valid annotations after remapping.")
                    # else: # Optional: print if an image doesn't have a corresponding label file
                    #    print(f"Info: Label file not found for image {img_file}, copied image only.")


    combined_yaml_path = combined_path / 'data.yaml'
    combined_yaml_content = {
        'path': str(combined_path.resolve()), # Root path of the dataset
        'train': str(Path('train') / 'images'), # Relative to 'path'
        'val': str(Path('valid') / 'images'),   # Relative to 'path'
        'test': str(Path('test') / 'images'),  # Relative to 'path'
        'nc': len(unified_classes),
        'names': unified_classes
    }

    with open(combined_yaml_path, 'w') as f:
        yaml.dump(combined_yaml_content, f, default_flow_style=None, sort_keys=False)

    print(f"\u2705 Combined dataset created at: {combined_path.resolve()}")
    print(f"\u2705 With {len(unified_classes)} unified classes: {unified_classes}")
    print(f"\U0001F4C1 Total image files processed: {file_count}")

    return combined_path, unified_classes

combined_dataset_path, classes = combine_datasets(dataset1_obj, dataset2_obj)

# =============================================================================
# STEP 3: Train YOLO Model
# =============================================================================

from ultralytics import YOLO

def train_warehouse_model(dataset_yaml_path, class_list):
    if not dataset_yaml_path or not Path(dataset_yaml_path).exists():
        print("\U0001F6AB Training cannot start: Combined dataset YAML not found.")
        return None, None
    if not class_list:
        print("\U0001F6AB Training cannot start: No classes defined for the model.")
        return None, None

    print("\U0001F680 Starting YOLO model training...")
    # Use a standard Ultralytics model, e.g., yolov8x.pt (large), yolov8n.pt (small)
    # If 'yolo11x.pt' is your custom pre-trained model or specific architecture, ensure it's correctly referenced.
    # For this example, we use 'yolov8x.pt'. Ultralytics will download it if not present.
    model = YOLO('yolov8x.pt') 

    training_args = {
        'data': str(dataset_yaml_path),
        'epochs': 35,       # Consider reducing for initial testing (e.g., 3-10 epochs)
        'imgsz': 640,
        'batch': -1,         # Adjust based on your GPU memory (e.g., 8, 4, or -1 for auto-batch)
        'device': 'auto',    # Use '0' for GPU 0, 'cpu', or 'auto'
        'patience': 15,      # Consider increasing if epochs are high
        'save': True,
        'cache': False,       # Set to True or 'ram'/'disk' if I/O is slow and you have resources
        'workers': 4,        # Adjust based on your CPU cores and dataloader performance
        'project': 'warehouse_monitoring_runs', # Changed project name slightly for clarity
        'name': 'yolo_warehouse_v1_run',       # Changed experiment name slightly
        'exist_ok': True,    # Allows re-running and overwriting previous experiment with same name
        'verbose': True,
        # 'logger': True, # Removed: Not a standard argument, logging is handled by Ultralytics
    }
    
    # Check if number of classes in the model matches the dataset
    # This is typically handled by YOLO if starting from a pretrained model with different nc,
    # but good to be aware of. The model's head will be reinitialized.
    
    results = model.train(**training_args)
    print("\u2705 Training completed!")
    
    # Save the path to the best model for later use
    # The actual path is inside results.save_dir or model.trainer.save_dir
    best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
    print(f"\U0001F4BE Best model saved at: {best_model_path}")
    
    return model, results, best_model_path


# Ensure combined_dataset_path and classes are valid before training
trained_model = None
training_results_obj = None
best_model_path_global = None

if combined_dataset_path and classes:
    data_yaml_for_training = combined_dataset_path / 'data.yaml'
    trained_model, training_results_obj, best_model_path_global = train_warehouse_model(data_yaml_for_training, classes)
else:
    print("\U0001F6AB Skipping training due to issues in dataset combination.")

# =============================================================================
# STEP 4: Evaluate Model Performance
# =============================================================================

def evaluate_model(model_to_evaluate, data_yaml_path_for_eval):
    if model_to_evaluate is None:
        print("\U0001F6AB No model trained. Skipping evaluation.")
        return None
    if not data_yaml_path_for_eval or not Path(data_yaml_path_for_eval).exists():
        print("\U0001F6AB Cannot evaluate: Dataset YAML for validation not found.")
        return None
        
    print("\U0001F4CA Evaluating model performance on the validation set...")
    try:
        # Ensure the model uses the correct data configuration for validation
        val_results = model_to_evaluate.val(data=str(data_yaml_path_for_eval), split='val')
        print(f"mAP50-95 (Box): {val_results.box.map:.4f}")
        print(f"mAP50 (Box): {val_results.box.map50:.4f}")
        # Precision and Recall might need to be accessed differently or might not be directly on box
        # For specific P, R:
        # print(f"Precision: {val_results.box.mp:.4f}") # mp might not exist directly like this
        # print(f"Recall: {val_results.box.mr:.4f}") # mr might not exist directly like this
        # Ultralytics typically prints these during validation. The results object contains detailed metrics.
        # Access general metrics
        print(f"All metrics: {val_results.metrics}") # Contains precision, recall, mAP etc.
        return val_results
    except Exception as e:
        print(f"\U0001F6AB Error during model evaluation: {e}")
        return None


if trained_model and combined_dataset_path:
    data_yaml_for_eval = combined_dataset_path / 'data.yaml'
    validation_results = evaluate_model(trained_model, data_yaml_for_eval)
else:
    print("\U0001F6AB Skipping model evaluation.")
    validation_results = None

# =============================================================================
# STEP 5: Test Model on Sample Images
# =============================================================================

def test_model_inference(model_to_test, dataset_base_path, num_images=5):
    if model_to_test is None:
        print("\U0001F6AB No model available. Skipping inference test.")
        return
    if not dataset_base_path or not Path(dataset_base_path).exists():
        print("\U0001F6AB Dataset path for testing not found. Skipping inference test.")
        return

    print("\U0001F50D Testing model inference...")
    test_img_dir = Path(dataset_base_path) / 'test' / 'images'
    
    if not test_img_dir.exists():
        print(f"\U0001F6AB Test images directory not found at: {test_img_dir}")
        return

    test_images = list(test_img_dir.glob('*.jpg')) + list(test_img_dir.glob('*.jpeg')) + list(test_img_dir.glob('*.png'))
    
    if not test_images:
        print(f"\U0001F6AB No test images found in {test_img_dir} with .jpg, .jpeg, or .png extensions.")
        return

    sample_test_images = test_images[:num_images]
    if not sample_test_images:
        print("\U0001F6AB Not enough test images to sample from.") # Should be caught by previous check too
        return

    results_list = model_to_test(sample_test_images) # Perform inference
    
    inference_output_dir = Path("./inference_results")
    inference_output_dir.mkdir(parents=True, exist_ok=True)

    for i, result in enumerate(results_list):
        print(f"--- Image {i+1}: {Path(sample_test_images[i]).name} ---")
        print(f"Detected {len(result.boxes)} objects")
        class_counts = {}
        if hasattr(model_to_test, 'names') and model_to_test.names:
            model_class_names = model_to_test.names
        else: # Fallback if model.names isn't populated as expected
            model_class_names = {k: f"class_{k}" for k in range(result.boxes.cls.max().item() + 1)} if len(result.boxes) > 0 else {}


        for box in result.boxes:
            class_id = int(box.cls)
            class_name = model_class_names.get(class_id, f"unknown_class_{class_id}")
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        print(f"Object counts: {class_counts}")
        
        # Save the image with detections
        save_path = inference_output_dir / f"result_{Path(sample_test_images[i]).stem}.jpg"
        result.save(filename=str(save_path))
        print(f"Saved result to {save_path}")
        
    print(f"\u2705 Inference results for {len(sample_test_images)} images saved in {inference_output_dir.resolve()}!")

if trained_model and combined_dataset_path:
    test_model_inference(trained_model, combined_dataset_path)
else:
    print("\U0001F6AB Skipping model inference test.")

# =============================================================================
# STEP 6: Export Model for Production
# =============================================================================

def export_trained_model(model_to_export, export_path_base):
    if model_to_export is None:
        print("\U0001F6AB No model trained. Skipping export.")
        return None
    if export_path_base is None or not Path(export_path_base).parent.exists(): # Check if parent of best.pt exists
        print(f"\U0001F6AB Export path base seems invalid ({export_path_base}). Cannot determine export directory. Skipping export.")
        return None

    print("\U0001F4E4 Exporting model...")
    try:
        # Exporting to ONNX format. Other formats: 'torchscript', 'coreml', 'engine', 'pb', etc.
        # The export path will be relative to the `save_dir` of the model, or can be specified.
        # If `model_to_export` is the model object from `train`, its `export` method handles paths well.
        # Let's ensure the export directory is based on the actual run.
        
        onnx_file_path = model_to_export.export(format='onnx', imgsz=640) # imgsz can be specified for export
        print(f"\u2705 Model exported successfully to ONNX format: {onnx_file_path}")
        
        # The .pt file is already saved (best.pt and last.pt) in the training run directory.
        # The `export_path_base` here refers to the `best.pt` path from training.
        export_dir = Path(export_path_base).parent # weights directory
        print(f"\U0001F4C1 Main model files location (.pt, .onnx): {export_dir.resolve()}")
        return onnx_file_path
    except Exception as e:
        print(f"\U0001F6AB Error during model export: {e}")
        return None

exported_onnx_path = None
if trained_model and best_model_path_global:
    exported_onnx_path = export_trained_model(trained_model, best_model_path_global)
else:
    print("\U0001F6AB Skipping model export.")


# =============================================================================
# STEP 7: Create Inference Function for Production
# =============================================================================

def warehouse_monitor(image_path, model_path=None, confidence_threshold=0.5):
    if model_path is None:
        print("\U0001F6AB Error: Model path not provided for warehouse_monitor.")
        # Fallback to a default expected path if best_model_path_global is set
        if best_model_path_global and Path(best_model_path_global).exists():
            print(f"Using global best model path: {best_model_path_global}")
            model_path = best_model_path_global
        else: # Try to construct the path from training args (less reliable if names changed)
            default_path = Path('warehouse_monitoring_runs') / 'yolo_warehouse_v1_run' / 'weights' / 'best.pt'
            print(f"Attempting to use default path: {default_path}")
            if default_path.exists():
                model_path = default_path
            else:
                print(f"\U0001F6AB Default model path {default_path} not found. Please specify a valid model_path.")
                return None
                
    if not Path(model_path).exists():
        print(f"\U0001F6AB Error: Model file not found at {model_path}")
        return None

    try:
        inference_model = YOLO(str(model_path)) # Load the specific model
    except Exception as e:
        print(f"\U0001F6AB Error loading model {model_path}: {e}")
        return None

    print(f"\U0001F52D Performing inference with model: {model_path} on image: {image_path}")
    results = inference_model(image_path, conf=confidence_threshold)

    detection_summary = {
        'image_path': str(image_path),
        'total_objects': 0,
        'class_counts': {},
        'detections': [] # List of {'class': name, 'confidence': conf, 'bbox': [x1,y1,x2,y2]}
    }

    if hasattr(inference_model, 'names') and inference_model.names:
        model_class_names = inference_model.names
    else: # Fallback if model.names isn't populated as expected after loading
        print("\U0001F6AB Warning: Model class names not found directly on loaded model. Will use 'class_ID' format.")
        model_class_names = {}


    for result in results: # Iterates over images, though here we pass one image path
        if result.boxes is not None:
            detection_summary['total_objects'] = len(result.boxes)
            for box in result.boxes:
                class_id = int(box.cls)
                # Try to get class name, fallback if model_class_names is empty or ID is missing
                class_name = model_class_names.get(class_id, f"class_{class_id}")
                
                confidence = float(box.conf)
                
                detection_summary['class_counts'][class_name] = \
                    detection_summary['class_counts'].get(class_name, 0) + 1
                
                detection_summary['detections'].append({
                    'class': class_name,
                    'confidence': round(confidence, 4),
                    'bbox': [round(coord, 2) for coord in box.xyxy.tolist()[0]] # x1, y1, x2, y2
                })
    return detection_summary

# =============================================================================
# Final Summary
# =============================================================================

print("\n" + "="*60)
print("\U0001F389 Warehouse AI Monitoring System Pipeline Complete! \U0001F389")
print("="*60 + "\n")

if trained_model and best_model_path_global:
    print("\U0001F527 Use the `warehouse_monitor(image_path, model_path)` function for production inference.")
    print(f"   Example: warehouse_monitor('path/to/your/image.jpg', model_path='{best_model_path_global}')\n")
    print("\U0001F4CA Model Performance Summary:")
    if classes:
        print(f"  - Classes configured for training: {len(classes)} {classes}")
    if training_results_obj:
        print(f"  - Training epochs completed: {training_results_obj.epochs if training_results_obj.epochs else training_args.get('epochs', 'N/A')}") # Access actual epochs if available
    print(f"  - Best model saved at: {best_model_path_global}")
    if exported_onnx_path:
        print(f"  - Model exported to ONNX at: {exported_onnx_path}")
    if validation_results and hasattr(validation_results, 'box'):
        print(f"  - Validation mAP50-95 (Box): {validation_results.box.map:.4f}")
        print(f"  - Validation mAP50 (Box): {validation_results.box.map50:.4f}")
else:
    print("\U0001F6AB Model training or setup was not fully completed. Please check logs.")
    if not combined_dataset_path or not classes:
         print("  - Issue: Dataset combination failed or yielded no classes.")
    elif not trained_model:
         print("  - Issue: Model training did not complete successfully.")


# Example of how to use the inference function (if a model was trained):
# if best_model_path_global and combined_dataset_path:
#     print("\n" + "="*30 + " EXAMPLE INFERENCE " + "="*30)
#     test_img_dir_for_example = combined_dataset_path / 'test' / 'images'
#     example_images = list(test_img_dir_for_example.glob('*.jpg')) + \
#                      list(test_img_dir_for_example.glob('*.jpeg')) + \
#                      list(test_img_dir_for_example.glob('*.png'))
#     if example_images:
#         example_image_path = example_images[0]
#         print(f"Running inference on example image: {example_image_path}")
#         summary = warehouse_monitor(image_path=str(example_image_path), model_path=str(best_model_path_global))
#         if summary:
#             print("Inference Summary:")
#             import json
#             print(json.dumps(summary, indent=2))
#     else:
#         print("No example images found in the test set to run a sample inference.")
# else:
#     print("\nSkipping example inference as model path or dataset path is not available.")

🔄 Downloading Dataset 1: Warehouse General...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Detection_box-1 in yolov8:: 100%|██████████| 4016/4016 [00:00<00:00, 5726.39it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Dataset 1 downloaded to: /kaggle/working/Detection_box-1
🔄 Downloading Dataset 2: Warehouse Object Detection...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Vikas_Warehouse_Obj_detection-10 in yolov8:: 100%|██████████| 12454/12454 [00:02<00:00, 4752.40it/s]


✅ Dataset 2 downloaded to: /kaggle/working/Vikas_Warehouse_Obj_detection-10
✅ Dataset download completed!
🔄 Combining datasets...
✅ Combined dataset created at: /kaggle/working/combined_warehouse_dataset
✅ With 7 unified classes: ['box', 'box_broken', 'forklift', 'open_package', 'package', 'pallets', 'person']
📁 Total image files processed: 8223
🚀 Starting YOLO model training...


...

Using Python 3.11.11 environment at: /usr
Resolved 22 packages in 308ms



requirements: AutoUpdate success ✅ 4.1s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...


 Downloaded onnxruntime-gpu
Prepared 4 packages in 3.25s
Installed 4 packages in 13ms
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnxruntime-gpu==1.22.0
 + onnxslim==0.1.53


ONNX: slimming with onnxslim 0.1.53...
ONNX: export success ✅ 11.5s, saved as 'warehouse_monitoring_runs/yolo_warehouse_v1_run/weights/best.onnx' (260.1 MB)

Export complete (15.3s)
Results saved to /kaggle/working/warehouse_monitoring_runs/yolo_warehouse_v1_run/weights
Predict:         yolo predict task=detect model=warehouse_monitoring_runs/yolo_warehouse_v1_run/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=warehouse_monitoring_runs/yolo_warehouse_v1_run/weights/best.onnx imgsz=640 data=combined_warehouse_dataset/data.yaml  
Visualize:       https://netron.app
✅ Model exported successfully to ONNX format: warehouse_monitoring_runs/yolo_warehouse_v1_run/weights/best.onnx
📁 Main model files location (.pt, .onnx): /kaggle/working/warehouse_monitoring_runs/yolo_warehouse_v1_run/weights

🎉 Warehouse AI Monitoring System Pipeline Complete! 🎉

🔧 Use the `warehouse_monitor(image_path, model_path)` function for production inference.
   Example: warehouse_monitor('p

AttributeError: 'DetMetrics' object has no attribute 'epochs'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class.
        names (dict): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (dict): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
    